# Process walkthrough:
- Access an excel file through user defined path
- Loop through each sheet to look for unique identifiers like 'Group Name:' to identify Oxford Plans of the Table 1 format using condition given in the `collecting tables` function
- Once Table 1 format is detected, check if there are multiple tables present in the sheet.
- If there are more than one table in a sheet, use the `separate_table` function to separate the tables and store them individually in a list
- If there is only 1 table in the sheet, again, store it in the same list 
- Loop over each table in the list and clean the data
- Once cleaned, concatenate the clean dfs into a final df

## Separating tables and Collecting them individually

In [147]:
import pandas as pd

#function to separate tables in case of multiple tables in a sheet

def separate_tables(df, df_list):
    # Convert all columns to string type
    df = df.astype(str)
    
    # Check if 'Group Name:' row exists in the DataFrame
    if df.apply(lambda x: x.str.contains('Group Name:')).any().any():
        # Find the row index where 'Group Name:' is present
        header_index = (df.apply(lambda x: x.str.contains('Group Name:')).any(axis=1)).idxmax()

        # Split the DataFrame into df1 and df2 based on the 'Group Name:' row
        df1 = df.iloc[:header_index]
        df2 = df.iloc[header_index + 1:]

        # Set the header of df2 using the 'Group Name:' row
        df2.columns = df.iloc[header_index]

        # Reset index for both DataFrames
        df1 = df1.reset_index(drop=True)
        df2 = df2.reset_index(drop=True)

        df_list.append(df1)
        df_list.append(df2)
        
    

#function to extract tables individually and store them in a list

def collecting_tables(file_path):
    # Read the Excel file
    xls = pd.ExcelFile(file_path)
    
    # Initialize an empty list to store dataframes
    table_list = []
    
    # Iterate through each sheet
    for sheet_name in xls.sheet_names:
        # Read the sheet into a DataFrame
        df = pd.read_excel(file_path, header=None, sheet_name=sheet_name)
        
        # Check if "Group Name:" exists in the top-left cells
        if "Group Name:" in df.iloc[:5, :5].values:
            df = df.T
            df.columns = df.iloc[0]
            df = df[1:]
            df.dropna(axis=0, how='all', inplace=True)
            df.reset_index(inplace=True, drop=True)
            
            df=df.astype(str)
            
            
            
            if (df.applymap(lambda x: 'Group Name:' in x)).any(axis=1).any():
                separate_tables(df, table_list)
            

                for table in table_list:
                    separate_tables(table, table_list)
                table_list = [item for item in table_list if not item.apply(lambda x: x.str.contains('Group Name:')).any().any()]
                
            else:
                table_list.append(df)
            
            
        else:
            pass
    
    
    return table_list


In [148]:
table_list=collecting_tables(r"C:\Users\shres\Downloads\multi table oxford sheet.xlsx")


C:\Users\shres\AppData\Local\Temp\ipykernel_36028\4097615813.py:45: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "Group Name:" in df.iloc[:5, :5].values:


In [149]:
len(table_list)

11

In [150]:
table_list[5]

,NaN,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Please Note: All items listed on this form are subject to review,NaN,NaN,General Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,In Network (IN),PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,"For Direct, EPO or Value Option Products",IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,Out of Network (OON):,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum,Rx Copays (OV/Spec Copay must be listed above),Rx Deductible- all drugs or brand only,Rx PLAN,"List all three parts of the benefit: visit or dollar limit, INN cost share and OON cost share"
0,New benefits- Scroll right,Tienam Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,nan,nan,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,nan,23,NJ,Large,HMO,Non Gated,Freedom,nan,30/45,150,IP: 0 after deductible OP: 0 after deductible,nan,2500/5000,100,3000/6000,3000/6001,nan,2500/5002,92,32000/50002,32000/50002,10/25/50,nan,nan,nan
1,nan,nan,nan,nan,nan,nan,nan,nan,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,nan,25,NJ,Large,HMO,Non Gated,Freedom,nan,30/50,160,IP: 0 after deductible OP: 0 after deductible,nan,2500/5000,90,32000/50000,32000/50000,nan,2500/5003,93,32000/50003,32000/50003,nan,nan,nan,nan
2,nan,nan,nan,nan,nan,nan,nan,nan,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,nan,45,NJ,Large,HMO,Non Gated,Freedom,nan,0/0,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5001,91,32000/50001,32000/50001,nan,2500/5004,94,32000/50004,32000/50004,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,nan,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,nan,30/55,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5002,92,32000/50002,32000/50002,nan,2500/5002,92,32000/50002,32000/50002,10/25/50,nan,nan,All radiology at 100%
4,nan,nan,nan,nan,nan,nan,nan,nan,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,nan,30/60,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5003,93,32000/50003,32000/50003,nan,2500/5003,93,32000/50003,32000/50003,nan,nan,nan,nan
5,nan,nan,nan,nan,nan,nan,nan,nan,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,nan,0/1,nan,IP: 0 after deductible OP: 0 after deductible,nan,2500/5004,94,32000/50004,32000/50004,nan,2500/5004,94,32000/50004,32000/50004,nan,nan,nan,nan
6,INC2197031 Updated in yellow,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
7,INC567870 8/9/21,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Cleaning the data

In [151]:
import pandas as pd
import numpy as np

def clean_data(df_list):
    dfs = []
    
    for df in df_list:
        # Replace 'nan' strings with actual NaN values
        df.replace('nan', np.nan, inplace=True)
        
        # Find the index of 'Group Name:' column
        group_name_index = df.columns.get_loc('Group Name:')
        
        # Remove columns before 'Group Name:' column
        df = df.iloc[:, group_name_index:]
        
        # Fill missing values in specific columns
        columns_to_fill = ['Group Name:', 'Group Number(or TBD)', 'Request Date:', 'Effective Date or NSB:']
        for col in columns_to_fill:
            df[col].fillna(df[col].iloc[0], inplace=True)
        
        # Rename columns to strings
        df.columns = [str(column) for column in df.columns]
        
        # Iterate through each column to check for 'Requested Benefits' in string columns
        for column in df.select_dtypes(include='object').columns:
            if any(df[column].apply(lambda x: 'Requested Benefits' in str(x))):
                df.rename(columns={column: 'Benefits Status'}, inplace=True)

        # Fill missing values in 'Benefits Status' column with forward fill
        if 'Benefits Status' in df.columns:
            df['Benefits Status'] = df['Benefits Status'].fillna(method='ffill')

        # Rename 'General Information' column to 'Plan Information'
        df.rename(columns={'General Information': 'Plan Information'}, inplace=True)
        
        # Remove columns where all values are null
        df = df.dropna(axis=1, how='all')
        
        # Set display option to show all columns
        pd.set_option('display.max_columns', None)
        
        dfs.append(df.reset_index(drop=True))  # Reset index of each DataFrame
        
    return dfs

# Example usage:
clean_dfs = clean_data(table_list)


In [152]:
len(clean_dfs)

11

### This is how each df looks like now


In [153]:
clean_dfs[8]

,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum,Rx Copays (OV/Spec Copay must be listed above),"List all three parts of the benefit: visit or dollar limit, INN cost share and OON cost share",Rx Copays (OV/Spec Copay must be listed above),"List all three parts of the benefit: visit or dollar limit, INN cost share and OON cost share"
0,XYZ Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,NaN,23,NJ,Large,HMO,Non Gated,Freedom,30/45,150,IP: 0 after deductible OP: 0 after deductible,2500/5000,100,3000/6000,3000/6001,2500/5002,92,32000/50002,32000/50002,10/25/50,NaN,10/25/50,NaN
1,XYZ Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,NaN,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003,NaN,NaN,NaN,NaN
2,XYZ Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,NaN,45,NJ,Large,HMO,Non Gated,Freedom,0/0,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004,NaN,NaN,NaN,NaN
3,XYZ Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002,10/25/50,All radiology at 100%,10/25/50,All radiology at 100%
4,XYZ Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,30/60,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5003,93,32000/50003,32000/50003,2500/5003,93,32000/50003,32000/50003,NaN,NaN,NaN,NaN
5,XYZ Inc,554556,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,0/1,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5004,94,32000/50004,32000/50004,2500/5004,94,32000/50004,32000/50004,NaN,NaN,NaN,NaN


## Concatenating All the dfs

In [154]:
def combine_data(clean_dfs):
    # Check for duplicate column names in each DataFrame
    for i, df in enumerate(clean_dfs):
        if df.columns.duplicated().any():
            # Drop duplicate columns
            df = df.loc[:, ~df.columns.duplicated()]
            clean_dfs[i] = df  # Update the DataFrame in the list

    # Concatenate the DataFrames
    concatenated_df = pd.concat(clean_dfs, ignore_index=True)
    return concatenated_df




# Example usage:
final_df = combine_data(clean_dfs)
final_df



,Group Name:,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product,Access,Network,PCP/Specialist OV Copay,ER Cost Share,Hospital Cost Share,IN Deductible,IN Coinsurance %,IN Coinsurance Limit,IN Out of Pocket Maximum,OON Deductible,OON Coinsurance %,OON Coinsurance Limit,OON Out of Pocket Maximum,Rx Copays (OV/Spec Copay must be listed above),"List all three parts of the benefit: visit or dollar limit, INN cost share and OON cost share"
0,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 1: P0145445 BUY UP,Freedom HMO 2323,NaN,23,NJ,Large,HMO,Non Gated,Freedom,30/45,150,IP: 0 after deductible OP: 0 after deductible,2500/5000,100,3000/6000,3000/6001,2500/5002,92,32000/50002,32000/50002,10/25/50,NaN
1,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,NaN,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003,NaN,NaN
2,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,NaN,45,NJ,Large,HMO,Non Gated,Freedom,0/0,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004,NaN,NaN
3,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002,10/25/50,All radiology at 100%
4,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,30/60,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5003,93,32000/50003,32000/50003,2500/5003,93,32000/50003,32000/50003,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,ABCDEF Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 2: P0145445 BUY UP,Freedom HMO 2324,NaN,25,NJ,Large,HMO,Non Gated,Freedom,30/50,160,IP: 0 after deductible OP: 0 after deductible,2500/5000,90,32000/50000,32000/50000,2500/5003,93,32000/50003,32000/50003,NaN,NaN
64,ABCDEF Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Benefits Currently in Place,Current Plan 3: P0145445 BUY UP,Freedom HMO 2325,NaN,45,NJ,Large,HMO,Non Gated,Freedom,0/0,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5001,91,32000/50001,32000/50001,2500/5004,94,32000/50004,32000/50004,NaN,NaN
65,ABCDEF Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,30/55,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5002,92,32000/50002,32000/50002,2500/5002,92,32000/50002,32000/50002,10/25/50,All radiology at 100%
66,ABCDEF Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,30/60,NaN,IP: 0 after deductible OP: 0 after deductible,2500/5003,93,32000/50003,32000/50003,2500/5003,93,32000/50003,32000/50003,NaN,NaN


In [155]:
import numpy as np
import pandas as pd

def filter_requested_benefits(final_df):
    final_df = final_df[final_df.apply(lambda row: 'Requested Benefits' in row.values, axis=1)]
    return final_df

def split_special_columns(final_df):
    column_to_split = 'PCP/Specialist OV Copay'
    if column_to_split in final_df.columns:
        split_cols = final_df[column_to_split].str.split('/', expand=True)
        final_df['PCP OV Copay'] = split_cols[0].astype(str).str.strip()
        final_df['Specialist OV Copay'] = split_cols[1].astype(str).str.strip()
        final_df.drop(columns=[column_to_split], inplace=True)
    return final_df
    

def split_columns(final_df):
    for col in final_df.columns:
        if final_df[col].apply(lambda x: '/' in str(x)).any():
            split_cols = final_df[col].str.split('/', expand=True)
            final_df[f'IND {col}'] = split_cols[0].astype(str).str.strip()
            final_df[f'FAMILY {col}'] = split_cols[1].astype(str).str.strip()
            final_df.drop(columns=[col], inplace=True)
    return final_df

def rename_columns(final_df):
    final_df = final_df.rename(columns={
        'Group Name:': 'Group Name',
        'FAMILY OON Deductible': 'FAMILY DEDUCTIBLE OON',
        'IND OON Deductible': 'IND DEDUCTIBLE OON',
        'FAMILY IN Deductible': 'FAMILY DEDUCTIBLE INN',
        'IND IN Deductible': 'IND DEDUCTIBLE INN',
        'FAMILY OON Out of Pocket Maximum': 'FAMILY OOP OON',
        'FAMILY IN Out of Pocket Maximum': 'FAMILY OOP INN',
        'IND IN Out of Pocket Maximum': 'IND OOP INN',
        'IND OON Out of Pocket Maximum': 'IND OOP OON',
        'Product': 'Product Name',
        'PCP OV Copay': 'PCP COPAY',
        'ER Cost Share': 'EMERGENCY ROOM COPAY',
        'Specialist OV Copay': 'SPEC COPAY',
        'Network': 'Network Description',
        'IN Coinsurance %': 'INN COINSURANCE',
        'OON Coinsurance %': 'OON COINSURANCE'
    })
    return final_df

# Assuming final_df is already populated
final_df = filter_requested_benefits(final_df)
final_df = split_special_columns(final_df)
final_df = split_columns(final_df)
final_df = rename_columns(final_df)

final_df

C:\Users\shres\AppData\Local\Temp\ipykernel_36028\3027659184.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['PCP OV Copay'] = split_cols[0].astype(str).str.strip()
C:\Users\shres\AppData\Local\Temp\ipykernel_36028\3027659184.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Specialist OV Copay'] = split_cols[1].astype(str).str.strip()
C:\Users\shres\AppData\Local\Temp\ipykernel_36028\3027659184.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

,Group Name,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product Name,Access,Network Description,EMERGENCY ROOM COPAY,Hospital Cost Share,INN COINSURANCE,OON COINSURANCE,"List all three parts of the benefit: visit or dollar limit, INN cost share and OON cost share",PCP COPAY,SPEC COPAY,IND DEDUCTIBLE INN,FAMILY DEDUCTIBLE INN,IND IN Coinsurance Limit,FAMILY IN Coinsurance Limit,IND OOP INN,FAMILY OOP INN,IND DEDUCTIBLE OON,FAMILY DEDUCTIBLE OON,IND OON Coinsurance Limit,FAMILY OON Coinsurance Limit,IND OOP OON,FAMILY OOP OON,IND Rx Copays (OV/Spec Copay must be listed above),FAMILY Rx Copays (OV/Spec Copay must be listed above)
3,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92,92,All radiology at 100%,30,55,2500,5002,32000,50002,32000,50002,2500,5002,32000,50002,32000,50002,10,25
4,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,93,93,NaN,30,60,2500,5003,32000,50003,32000,50003,2500,5003,32000,50003,32000,50003,nan,nan
5,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,94,94,NaN,0,1,2500,5004,32000,50004,32000,50004,2500,5004,32000,50004,32000,50004,nan,nan
9,Tesla Inc,554658,2021-04-02 00:00:00,2021-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92,92,All radiology at 100%,30,55,2500,5002,32000,50002,32000,50002,2500,5002,32000,50002,32000,50002,10,25
10,Tesla Inc,554658,2021-04-02 00:00:00,2021-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,93,93,NaN,30,60,2500,5003,32000,50003,32000,50003,2500,5003,32000,50003,32000,50003,nan,nan
11,Tesla Inc,554658,2021-04-02 00:00:00,2021-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,94,94,NaN,0,1,2500,5004,32000,50004,32000,50004,2500,5004,32000,50004,32000,50004,nan,nan
15,Tesla Inc,554658,2022-04-02 00:00:00,2022-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92,92,All radiology at 100%,30,55,2500,5002,32000,50002,32000,50002,2500,5002,32000,50002,32000,50002,10,25
16,Tesla Inc,554658,2022-04-02 00:00:00,2022-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,93,93,NaN,30,60,2500,5003,32000,50003,32000,50003,2500,5003,32000,50003,32000,50003,nan,nan
17,Tesla Inc,554658,2022-04-02 00:00:00,2022-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,94,94,NaN,0,1,2500,5004,32000,50004,32000,50004,2500,5004,32000,50004,32000,50004,nan,nan
21,Tesla Inc,554658,2023-04-02 00:00:00,2023-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92

In [156]:
# Assuming final_df is already populated
final_df = filter_requested_benefits(final_df)
final_df = split_special_columns(final_df)
final_df = split_columns(final_df)
final_df = rename_columns(final_df)

In [157]:
final_df

,Group Name,Group Number(or TBD),Request Date:,Effective Date or NSB:,Benefits Status,Plan Information,CSP,Base (Similar standard plan) Tracking ID,No of Employees Enrolled:,"Group State (NY,NJ,CT)",Market (Large or Small),Product Name,Access,Network Description,EMERGENCY ROOM COPAY,Hospital Cost Share,INN COINSURANCE,OON COINSURANCE,"List all three parts of the benefit: visit or dollar limit, INN cost share and OON cost share",PCP COPAY,SPEC COPAY,IND DEDUCTIBLE INN,FAMILY DEDUCTIBLE INN,IND IN Coinsurance Limit,FAMILY IN Coinsurance Limit,IND OOP INN,FAMILY OOP INN,IND DEDUCTIBLE OON,FAMILY DEDUCTIBLE OON,IND OON Coinsurance Limit,FAMILY OON Coinsurance Limit,IND OOP OON,FAMILY OOP OON,IND Rx Copays (OV/Spec Copay must be listed above),FAMILY Rx Copays (OV/Spec Copay must be listed above)
3,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92,92,All radiology at 100%,30,55,2500,5002,32000,50002,32000,50002,2500,5002,32000,50002,32000,50002,10,25
4,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,93,93,NaN,30,60,2500,5003,32000,50003,32000,50003,2500,5003,32000,50003,32000,50003,nan,nan
5,Tesla Inc,554658,2020-04-02 00:00:00,2020-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,94,94,NaN,0,1,2500,5004,32000,50004,32000,50004,2500,5004,32000,50004,32000,50004,nan,nan
9,Tesla Inc,554658,2021-04-02 00:00:00,2021-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92,92,All radiology at 100%,30,55,2500,5002,32000,50002,32000,50002,2500,5002,32000,50002,32000,50002,10,25
10,Tesla Inc,554658,2021-04-02 00:00:00,2021-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,93,93,NaN,30,60,2500,5003,32000,50003,32000,50003,2500,5003,32000,50003,32000,50003,nan,nan
11,Tesla Inc,554658,2021-04-02 00:00:00,2021-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,94,94,NaN,0,1,2500,5004,32000,50004,32000,50004,2500,5004,32000,50004,32000,50004,nan,nan
15,Tesla Inc,554658,2022-04-02 00:00:00,2022-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92,92,All radiology at 100%,30,55,2500,5002,32000,50002,32000,50002,2500,5002,32000,50002,32000,50002,10,25
16,Tesla Inc,554658,2022-04-02 00:00:00,2022-01-03 00:00:00,Requested Benefits,Current Plan 2: P0145445 BUY UP,Freedom HMO 2327,KACTOHP58- MNS000026,26,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,93,93,NaN,30,60,2500,5003,32000,50003,32000,50003,2500,5003,32000,50003,32000,50003,nan,nan
17,Tesla Inc,554658,2022-04-02 00:00:00,2022-01-03 00:00:00,Requested Benefits,Current Plan 3: P0145445 BUY UP,Freedom HMO 2328,KACTOHP58- MNS000027,15,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,94,94,NaN,0,1,2500,5004,32000,50004,32000,50004,2500,5004,32000,50004,32000,50004,nan,nan
21,Tesla Inc,554658,2023-04-02 00:00:00,2023-01-03 00:00:00,Requested Benefits,Current Plan 1: P0145445 BUY UP,Freedom HMO 2326,KACTOHP58- MNS000025,36,NJ,Large,HMO,Non Gated,Freedom,NaN,IP: 0 after deductible OP: 0 after deductible,92

### Notes:

- We need to consider cases where tables are arranged top to bottom. Presently the `separate_table` function only deals with tables arranged left to right

- We need to include the logic to get rid additional/irrelevant texts present in the excel sheet above or below the tables

- Some sheets contain 'Corrected Values', written next to the incorrect values which need to be considered.
